## Research Log 12/16/19-12/19/19

## 12/16/19 Mon
- The qiskit infrastructure for VQE has a built-in, parallelized procedure for computing the mean and the variance of the Hamiltonian expectation value from the variational state
- Internally, this is done with 
```python
    VQE._energy_evaluation(...),
```
which itself relies on 
```python
    WeightedPauliOperator.evaluate_with_result(...).
```
The above function *does not* return the standard deviation, but that information can be accessed via a callback function that is specified upon initializing the `VQE` class.

- I need to check, however, whether the variance and standard deviation calculations inherired from `WeightedPauliOperator` are still correct when the Paulis are grouped (probably not). Then the method has to be made virtual and modified.
- I also want to know more about how the `SPSA` optimizer works. In particular, how to decrease `eval_counts`. From what my `simple_callback` has reported, the convergence happens pretty quickly and just oscillates for a long time. There is probably some tolerance parameter.
- Finally, and very importantly, I need to figure out how to generalize my variational form for an arbitrary Schwinger Hamiltoninan matrix, or, at first, for an easier case of a $2^q \times 2^q$ matrix.
- Here is the result running VQE with for a $2$-spatial-site Hamiltonian in the even-parity sector, now with native standard deviation calculation):
```
Energy evaluation 1 returned -0.258369 +/- 0.036501
Energy evaluation 10 returned -0.111133 +/- 0.036235
Energy evaluation 100 returned -0.819157 +/- 0.040132
Energy evaluation 1000 returned -1.006818 +/- 0.034266
Energy evaluation 2051 returned -1.027508 +/- 0.031229
```
- Post-run calls to `VQE._energy_evaluation` are stochastic: circuits are constructed again (**possibly transpiled? --- need to check**), and then measured (**how many times?**). A number of calls one after another returned the following:
```
Energy evaluation 2059 returned -1.041773 +/- 0.030909
Energy evaluation 2060 returned -1.029380 +/- 0.030883
Energy evaluation 2061 returned -1.002505 +/- 0.031510
Energy evaluation 2062 returned -1.029749 +/- 0.030635
Energy evaluation 2063 returned -1.004319 +/- 0.031662
```
- Compare the above with the value I get from exact diagonalization in Mathematica: **-1.01162**.

## 12/17/19 Tue
To-Do:
1. How are the circuts measured in ``VQE._energy_evaluation``? How many shots? How do I change it?
2. Is the calculation of variance and standard deviation still correct when the Paulis are grouped? Reference the IBM paper.
3. Is there a way to speed up SPSA? How does SPSA work generally? --- Could decrease max steps. Not much control beyond that.

To work with qiskit more systematically: Would like to fork a repo and make changes to code.
- Forked a repository and created a new branch for physics. Eventually I'll add all of my classes there in a structured way, but for now it's a distraction. The current upshot is that I get to modify code directly.

``VQE._energy_evaluation`` will jobs (and hence, measurements) via ``VQE._quantum_instance``; in turn, this object specifies it's run configuration via a RunConfig object that can be accessed as ``sim.quantum_instance.run_config``. Presently it returns 
```python
RunConfig(max_credits=10, shots=1024)
```
Which means that during optimization runs, jobs do not contain the whole memory output. This can be changed by running
```python
VQE.quantum_instance.set_config(memory=True)
```
after which ``run_config`` returns 
```python
RunConfig(max_credits=10, shots=1024, memory=True)
```
However, this is not necessary for estimating errors if that is done with the built-in ``_energy_evaluation`` function. Changing the number of shots to $8 * 1024$ and running ``_energy_evaluation`` again, I see the expected decrease in standard deviation by about $\sqrt{8} \approx 2.8$:
```
Energy evaluation 2057 returned -1.010090 +/- 0.010971
Energy evaluation 2058 returned -1.000480 +/- 0.010910
Energy evaluation 2059 returned -1.018623 +/- 0.010833
Energy evaluation 2060 returned -0.998235 +/- 0.011022
Energy evaluation 2061 returned -1.009337 +/- 0.010775
```

- Okay, so just need to **understand how the unbound method ``WeightedPauliOperator._routine_compute_mean_and_var`` works.** It's relieving that this function is ``parallel_map``'ed over a list whose each entry looks like:
```python
(Paulis, ...), counts
```
where the tuple contains Pauli objects bound to the same circuit, with the same counts results. Each Pauli object specifies a Pauli matrix and a coefficient with which that Pauli enters the computational basis expansion of the operator. The way the information is structured makes me think that the variance calculation is in fact done correctly --- this is to be checked later today.

- Would like to have a bit of a better idea what these Pauli objects are.

- Instead of bootstrapping, ``WeightedPauliOperator._routine_compute_mean_and_var`` computes the standard deviation of the hamiltonian from the covariance of the measurements. Everything matches Eq.(17-18) in the IBM paper [1704.05018v2](https://arxiv.org/pdf/1704.05018.pdf) --- errors are calculated correctly.

- One thing I don't understand is whether it is possible to estimate the uncertainty on the variational parameters, $\Delta \vec{\theta}$. From the description of the algorithm there does not seem to be a clear way to do that (*algorithmic uncertainty*)
- However, some information on the experimental uncertainty of those angles is of course hardware dependent: see Fig. S3 in 1704.05018v2.

Excerpt from the paper on measuring the Hamiltonian:

**The individual Pauli operators are measured by correlating
measurement outcomes of single-qubit dispersive readouts in the Z basis, which can be done simultaneously since
each qubit is provided with an individual readout resonator. In case a target multi-qubit Pauli operator contains non
diagonal single-qubit Pauli operator, single-qubit rotations (post-rotations) are performed before the measurement in
the Z basis. Specifically, a −π/2(π/2) rotation along the X(Y ) axis to measure a Y(X) single-qubit Pauli operator.**

## 12/18/19 Wed
To-Do
1. **How to generalize my variational form for an arbitrary Schwinger Hamiltoninan matrix? Or, at first, how to generalize this to an easier case of a $2^q\times2^q$ matrix?**
2. Work on qubit ordering change between Hamiltonian in IBM and textbook conventions, as well as how to explain it.

Another source of qubit mismatch may be the fact that I order the states in exact diagonalization in their value under $J^2$ operator? I need to check this.


- Before qubit ordering is fixed, I need to get uncertainties on the estimates of these coefficients, not just the energy estimate (energy estimate is not useful for us anyways).

- **Left column is from exact diagonalization, right column is from Qiskit -- note qubit ordering convention differences:**

```mathematica
{"0.7041621406145364`", "0.69921875`"},
{"0.2582831083289985`", "0.0390625`"},
{"0.03524433188695736`", "0.259765625`"},
{"0.002310419169507747`", "0.001953125`"}
```

- These uncertainties can be bootstrapped (different run so not exactly the right column):

```python
[(0.6918535232543945, 0.015006468722540236),
 (0.023607254028320312, 0.004773143187929709),
 (0.2806539535522461, 0.014563374693483086),
 (0.0038852691650390625, 0.0019066378554630945)]
```

- Spent a lot of time writing a recursive function to generate bistrings :)
- Confirmed my guess for qubit ordering in qiskit: see [1903.04359](https://arxiv.org/pdf/1903.04359.pdf), bottom of pg. 6. This means that the outcomes that I get from qiskit simply have to be inverted to get to the textbook ordering: e.g., '000' -> '000', '001' -> '100', and so on.
- Qubit ordering fixed!
- (**There is an important distinction on the uncertainty of the measurement vs. the uncertainity in the variational parameter**). If I run VQE with 1024 shots per measurement and then try to bootstrap uncertainties *at determined variational controls* with more shots per measurement, I have low variance and hidden bias.

- Would like to do a transpiler tutorial with Qiskit, this may provide a concrete way to generalize the construction of the variatonal form for an arbitrary number of qubits.

## 12/19/19 Thu
To-Do:
1. Outline what we want to do with the interpolating operator measurements.
2. Plot results with uncertainties. Make it pretty!
    - For even and odd Hamiltonian with 2 qubits (but different truncations)
    - Energy estimates of the Hamiltonian at that truncation. Ultimately also need a reference for how biased that measurement is (continuum limit, infinite volume, full Hilbert space with no truncation)
    - Uncertainties on the probabilities and coefficients vs exact diagonalization, with more or less truncation. 
3. Which direction to take next? 
    - Tweak details of error propagation, define for more qubits / arbitrary qubits and optimize.
    - Attempt constructing a transition operator on 2 qubits? (don't expect good performance yet from Mathematica)
4. Talk to P.&R. at 4.30 pm
5. Try transpiling the exact rotation matrix to see how it is transpiled. It should be easier to map an exact rotation matrix a gate and have it transpiled. If that actually yields a good circuit, then it's good enough for now.

### Measuring the transition operators: notes from 10/01/19, talking with Scott from Maryland
- Trick for measuring \\( \langle y | O | x \rangle \\) on QC: use ancilla to prepare (a properly normalized version of):
\\[
    | u \rangle 
        = \dfrac{1}{\sqrt{2}} \left( 
            | x \rangle + | y \rangle 
            \right)
\\]
\\[
    | v \rangle 
        = \dfrac{1}{\sqrt{2}} \left( 
            | x \rangle - | y \rangle
            \right)
\\]
and then measure \\( \langle u | O | u \rangle \\) and \\( \langle v | O | v \rangle \\).

- Then

\begin{align}
    \langle u | O | u \rangle - \langle v | O | v \rangle 
        &= \langle x | O | y \rangle + \langle y | O | x \rangle \\
\end{align}

- If $O = O_H + O_{SH}$ with $O_H = \frac{1}{2}\left(O + O^\dagger \right)$ and $O_{SH} = \frac{1}{2}\left(O - O^\dagger \right)$ so that $O_H$ is Hermitian and $O_{SH}$, skew-Hermitian,

\begin{align}
    \langle u | O | u \rangle - \langle v | O | v \rangle 
        &= 2 \left(\operatorname{Re}\left(O_H\right)+ i \operatorname{Im}\left(O_{SH}\right)\right) \\
    \langle u | O^\dagger | u \rangle - \langle v | O^\dagger | v \rangle 
        &= 2 \left(\operatorname{Re}\left(O_H\right)- i \operatorname{Im}\left(O_{SH}\right)\right) \\
\end{align}

- The above gives $\operatorname{Re}(O_H)$ and $\operatorname{Im}(O_{SH})$. 
- But how to prepare equal mixtures of multi-qubit states?
\begin{align}
    | u \rangle = \dfrac{1}{\sqrt{2}}\left(| x \rangle | 0 \rangle + | 0 \rangle | y \rangle\right)
        \neq | x \rangle | y \rangle
\end{align}

(evident from computing the density matrix). Perhaps the equal superposition can be similarly variationally prepared?

- Another thing: since the operators O have to be diagonal in the computational basis, you do not need to apriopri construct a variational basis. The QC will give you an optimal combination of diagonal operators that maximize the overlap. 

### Conversation with Natalie in December:

- Using an ancilla, it's possible to measure the inner product between two states. Here's a relevant [post](https://quantumcomputing.stackexchange.com/questions/3965/how-can-i-calculate-the-inner-product-of-two-quantum-registers-of-different-size) (references therein)

### Sanity check

- If O is a transition operator to a different symmetry sector, it must be that $\langle x | O |x \rangle = 0$ and $\langle y | O^\dagger | y \rangle = 0$.


### Qiskit-Codes for VQE with errors

![Even image](even.png)

![Odd image](odd.png)

## Some excerpts from notes on transpilation
- As a product of three CNOT gates, SWAP gates are expensive operations to perform on a noisy quantum devices. However, such operations are usually necessary for embedding a circuit into the limited entangling gate connectivities of actual devices. Thus, minimizing the number of SWAP gates in a circuit is a primary goal in the transpilation process.

